# 0. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Make Detections

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 3. Calculate Angles

In [52]:
shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
mouth_l = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].z]
mouth_r = [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].z]
nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,landmarks[mp_pose.PoseLandmark.NOSE.value].z]

In [89]:
def calculate_deviation(shoulder_l,shoulder_r,mouth_l,mouth_r,nose,base_shoulder_l,base_shoulder_r,base_mouth_l,base_mouth_r,base_nose):
    nose_f = abs(base_nose[0] - nose[0]) + abs(base_nose[1] - nose[1]) + abs(
            base_nose[2] - nose[2])
    print(nose_f)
    mouth_l_f = abs(base_mouth_l[0]- mouth_l[0]) + abs(
        base_mouth_l[1]- mouth_l[1]) + abs(base_mouth_l[2]- mouth_l[2])
    mouth_r_f = abs(base_mouth_r[0]- mouth_r[0]) + abs(
        base_mouth_r[1]- mouth_r[1]) + abs(base_mouth_r[2]- mouth_r[2])
    shoulder_l_f = abs(base_shoulder_l[0]- base_shoulder_l[0]) + abs(
        base_shoulder_l[1]- shoulder_l[1]) + abs(base_shoulder_l[2]- shoulder_l[2])
    shoulder_r_f = abs(base_shoulder_r[0]- base_shoulder_r[0]) + abs(
        base_shoulder_r[1]- shoulder_r[1]) + abs(base_shoulder_r[2]- shoulder_r[2])
    deviation = int(
        ((nose_f + mouth_l_f + mouth_r_f + shoulder_l_f + shoulder_r_f) / (base_nose[0]+
         base_mouth_l[0] + base_mouth_r[0] + base_shoulder_l[0] +
         base_shoulder_r[0]) * 25))
    return deviation
print(int(1-1))
     

0


# 4. DEVIATION Counter

In [90]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            mouth_l = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].z]
            mouth_r = [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y,landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].z]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,landmarks[mp_pose.PoseLandmark.NOSE.value].z]
            if counter == 0:
                base_shoulder_r = shoulder_r
                base_shoulder_l = shoulder_l
                base_mouth_l = mouth_l
                base_mouth_r = mouth_r
                base_nose = nose
            # Calculate angle
            deviation = calculate_deviation(shoulder_l, shoulder_r,mouth_l,mouth_r,nose,base_shoulder_l,base_shoulder_r,base_mouth_l,base_mouth_r,base_nose)
            # Visualize angle
            cv2.putText(image, str(deviation),  
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if deviation > 25:
                stage = "seggem imanik"
            elif deviation > 30:
                stage="nnighak seggem imanik"
            elif deviation > 40:
                stage = "ettes kan akhirak"                       
        except:
            pass
        counter +=1
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, stage, (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image,str(deviation),
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, stage, (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

0.0
0
0.2083994746208191
6
0.20958536863327026
7
0.2093227505683899
8
0.3822692632675171
14
0.3426765203475952
12
0.5773046016693115
22
0.4811667799949646
18
0.6926336884498596
26
0.7393396496772766
27
0.7174330949783325
26
0.6880366206169128
26
0.8390330672264099
31
0.8323546051979065
31
0.8909832835197449
34
0.8917635083198547
34
0.9108245968818665
35
0.9260311722755432
35
0.9289207458496094
35
0.8957248330116272
34
0.8737745881080627
33
0.7053021192550659
27
0.6670543551445007
25
0.681576669216156
26
0.6973732709884644
26
0.7026354670524597
27
0.7653622031211853
29
0.7379912734031677
28
0.7326356172561646
28
0.734889566898346
28
0.7340057492256165
28
0.754036545753479
29
0.7232345938682556
27
0.727755069732666
26
0.724622368812561
26
0.705479621887207
25
0.7233182191848755
26
0.6639755964279175
24
0.6522361636161804
24
0.5528039336204529
20
0.38154786825180054
14
0.3307644724845886
12
0.3448924422264099
12
0.28826451301574707
10
0.2442079782485962
9
0.0390164852142334
2
0.0466576814

KeyboardInterrupt: 